In [1]:
# coding=utf-8
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

options = FirefoxOptions()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)
# driver = webdriver.Firefox()

first = True


def add_info(url):
    global first
    driver.get(url)
    wait = WebDriverWait(driver, 20)
    if "gehalt.de" in url:
        wait.until(lambda driver: driver.current_url != url)
    link = driver.current_url
    if "stepstone.de" not in link:
        return (
            link,
            "Nicht stepstone",
            "Nicht stepstone",
            "Nicht stepstone",
            "Nicht stepstone",
            "Nicht stepstone",
            "Nicht stepstone",
        )
    # print(link)
    if first:
        wait.until(
            lambda driver: len(
                driver.find_elements(
                    By.CSS_SELECTOR,
                    "#ccmgt_explicit_preferences.privacy-prompt-button.secondary-button.options-button",
                )
            )
            > 0
        )
        driver.find_element(
            By.CSS_SELECTOR,
            "#ccmgt_explicit_preferences.privacy-prompt-button.secondary-button.options-button",
        ).click()
        driver.find_element(
            By.CSS_SELECTOR, ".secondary-button.ccmgt_reject_button"
        ).click()
        first = False

    if len(driver.find_elements(By.CLASS_NAME, "listing-content-provider-hw0nf9")) > 0:
        driver.find_element(By.CLASS_NAME, "listing-content-provider-hw0nf9").click()

    response = driver.page_source
    soup_object = BeautifulSoup(response)
    if len(soup_object.find_all(attrs={"data-genesis-element": "ALERT_CONTENT"})) > 0:
        return (
            link,
            "Stellenanzeige nicht mehr verfügbar",
            "Stellenanzeige nicht mehr verfügbar",
            "Stellenanzeige nicht mehr verfügbar",
            "Stellenanzeige nicht mehr verfügbar",
            "Stellenanzeige nicht mehr verfügbar",
            "Stellenanzeige nicht mehr verfügbar",
        )
    # Teilzeit und Remote
    if len(soup_object.find_all(class_="at-listing__list-icons_work-type")) > 0:
        work_type = soup_object.find(class_="at-listing__list-icons_work-type").find(
            attrs={"data-genesis-element": "TEXT"}
        )
        # # print(soup_object)
        # print(work_type)
        teilzeit_remote = work_type.text
    else:
        teilzeit_remote = "-"

    base = soup_object.find_all("div", class_="sc-EHOje jWEhHL")

    texts = ["", "", "", "", ""]

    for i in range(len(base)):
        area = base[i].find_all(["p", "li"])
        for element in area:
            texts[i] += element.text + "\n"

    if (teilzeit_remote == "-") & (texts == ["", "", "", "", ""]):
        errors = pd.read_pickle("../data/errors.pkl")
        idx = len(errors)
        errors.loc[idx] = link, str(soup_object)
        errors.to_pickle("../data/errors.pkl")

    return link, teilzeit_remote, texts[0], texts[1], texts[2], texts[3], texts[4]

In [2]:
import pandas as pd

jobs = pd.read_pickle("../data/jobs.pkl")
jobs_new = jobs[jobs["Teilzeit_Remote"].isna() == True]
jobs_new

,Titel,Unternehmen,Ort,Gehalt_min,Gehalt_max,JobID,Link,Datum,Teilzeit_Remote,Introduction,Description,Profile,We_offer,Contact
7810,(Junior) Performance Data Analyst (m/w/d),Intrum Deutschland GmbH,Essen,,,57791394,https://www.gehalt.de/stellenangebote-jobs/det...,2023-09-30,NaN,NaN,NaN,NaN,NaN,NaN
7811,Data Warehouse Developer / Architect*,"Joh. Berenberg, Gossler & Co. KG",Hamburg,,,57785037,https://www.gehalt.de/stellenangebote-jobs/det...,2023-09-30,NaN,NaN,NaN,NaN,NaN,NaN
7812,Data Architect Data Warehouse / Lakehouse (m/w/d),thyssenkrupp Steel Europe AG,Duisburg,,,57774714,https://www.gehalt.de/stellenangebote-jobs/det...,2023-09-30,NaN,NaN,NaN,NaN,NaN,NaN
7813,Data-Warehouse-Entwickler / ETL – Data Enginee...,Boerse Stuttgart Group,Stuttgart,4355,5806,57768373,https://www.gehalt.de/stellenangebote-jobs/det...,2023-09-30,NaN,NaN,NaN,NaN,NaN,NaN
7814,Data Engineer / Analyst (m/f/x),ZEISS,München,,,57782097,https://www.gehalt.de/stellenangebote-jobs/det...,2023-09-30,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8157,Application Manager & Business Analyst im Mark...,Baader Bank AG,Unterschleißheim bei München,3790,5887,57710455,https://www.gehalt.de/stellenangebote-jobs/det...,2023-09-30,NaN,NaN,NaN,NaN,NaN,NaN
8158,Business Analyst (m/w/d) Prozess- & Systemopti...,Continentale Lebensversicherung AG,München,5645,6048,57550334,https://www.gehalt.de/stellenangebote-jobs/det...,2023-09-30,NaN,NaN,NaN,NaN,NaN,NaN
8159,Senior Agile Business Analyst:in / Requirement...,HDI AG,"Hannover, Köln",5161,7419,57601937,https://www.gehalt.de/stellenangebote-jobs/det...,2023-09-30,NaN,NaN,NaN,NaN,NaN,NaN
8160,Business Analyst (m/w/d) im Squad „Optimierung...,SIGNAL IDUNA Gruppe,"Dortmund, Hamburg, Koblenz, Leipzig",3710,5484,57530536,https://www.gehalt.de/stellenangebote-jobs/det...,2023-09-30,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
jobs = pd.read_pickle("../data/jobs.pkl")
jobs1 = jobs[(jobs["Teilzeit_Remote"] == "-") & (jobs["Introduction"] == "")]
jobs1

,Titel,Unternehmen,Ort,Gehalt_min,Gehalt_max,JobID,Link,Datum,Teilzeit_Remote,Introduction,Description,Profile,We_offer,Contact
7534,Wissenschaftliche*r Mitarbeiter*in – Data Science,Fraunhofer-Einrichtung für Individualisierte u...,Lübeck,,,57751946,https://www.stepstone.de/stellenangebote--wiss...,2023-09-23,-,,,,,
7538,Working Student Data Science & Engineering for...,E.ON Energy Markets GmbH,Essen,,,57704417,https://www.stepstone.de/stellenangebote--work...,2023-09-23,-,,,,,
7547,Program and Project Manager Science and Innova...,Dr. Falk Pharma GmbH,Freiburg im Breisgau,4758,6371,57711983,https://www.stepstone.de/stellenangebote--prog...,2023-09-23,-,,,,,
7548,Projektleiter / Bauüberwachung (w/m/d) Life Sc...,Drees & Sommer SE,"Stuttgart, München, Ulm, Frankfurt, Hamburg, H...",,,57742707,https://www.stepstone.de/stellenangebote--proj...,2023-09-23,-,,,,,
7560,Ausbildung: Bachelor of Science (B.Sc.) Studie...,Robert Bosch GmbH,Schwäbisch Gmünd,,,57719606,https://www.stepstone.de/stellenangebote--ausb...,2023-09-23,-,,,,,
7561,Duales Studium für den Bachelor of Science (w/...,Deutsches Elektronen-Synchrotron DESY,Hamburg,,,57731356,https://www.stepstone.de/stellenangebote--dual...,2023-09-23,-,,,,,
7565,Duales Studium - Bachelor of Science - Wirtsch...,Novem Car Interior Design GmbH,Vorbach,,,57036048,https://www.stepstone.de/stellenangebote--dual...,2023-09-23,-,,,,,
7567,Werkstudent (w/m/d) Baumanagement für die Bran...,Drees & Sommer SE,Hannover,,,57463723,https://www.stepstone.de/stellenangebote--werk...,2023-09-23,-,,,,,
7568,Duales Studium (Bachelor of Science) Zentralba...,Deutsche Bundesbank,bundesweit,1465,1470,57228695,https://www.stepstone.de/stellenangebote--dual...,2023-09-23,-,,,,,
7569,Duales Studium Bachelor of Science (m/w/d) Bet...,Swiss Life Select Deutschland GmbH,"Köln, Bonn, Karlsruhe, Stuttgart, München, Ber...",,,57636276,https://www.stepstone.de/stellenangebote--dual...,2023-09-23,-,,,,,


In [3]:
indexes = jobs1.index
for i in indexes:
    jobs.loc[
        i,
        [
            "Link",
            "Teilzeit_Remote",
            "Introduction",
            "Description",
            "Profile",
            "We_offer",
            "Contact",
        ],
    ] = add_info(jobs1.loc[i, "Link"])
driver.quit()
jobs.to_pickle("../data/jobs.pkl")

In [4]:
jobs1 = jobs[(jobs["Teilzeit_Remote"] == "-") & (jobs["Introduction"] == "")]
jobs1

,Titel,Unternehmen,Ort,Gehalt_min,Gehalt_max,JobID,Link,Datum,Teilzeit_Remote,Introduction,Description,Profile,We_offer,Contact


In [13]:
errors = pd.read_pickle("../data/errors.pkl")
from bs4 import BeautifulSoup

soup_object = BeautifulSoup(errors.loc[0, "Link"])
print(soup_object.find_all(class_="at-listing__list-icons_work-type"))

[]


In [15]:
jobs.to_pickle("../data/jobs.pkl")

In [15]:
errors = errors[0:0]
errors.to_pickle("../data/errors.pkl")